In [1]:
import os
import numpy as np
import pandas as pd
from math import ceil

Import data and reduce memory usage. Taken from notebook [here](https://github.com/matthiasanderer/m5-accuracy-competition/blob/main/m5-simple-fe-evaluation.ipynb).

Global variables and helper functions

In [2]:
TARGET = 'sales'          # Our main target
END_TRAIN = 1913 + 28     # Last day in train set
MAIN_INDEX = ['id', 'd']  # We can identify item by these columns
ROOT_PATH = '/ssd003/projects/forecasting_bootcamp/bootcamp_datasets/m5-forecasting-accuracy'

def reduce_mem_usage(df):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64'] 
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    return df

def merge_by_concat(df1, df2, merge_on):
    merged_gf = df1[merge_on]
    merged_gf = merged_gf.merge(df2, on=merge_on, how='left')
    new_columns = [col for col in list(merged_gf) if col not in merge_on]
    df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
    return df1

Dataframes that uses less memory usage

In [ ]:
calendar_df = pd.read_csv(os.path.join(ROOT_PATH, 'calendar.csv'))
calendar_df = reduce_mem_usage(calendar_df)

prices_df = pd.read_csv(os.path.join(ROOT_PATH, 'sell_prices.csv'))
prices_df = reduce_mem_usage(prices_df)

train_df = pd.read_csv(os.path.join(ROOT_PATH, 'sales_train_evaluation.csv'))
train_df = train_df.drop(['d_{}'.format(str(i)) for i in range(1, 29)], axis=1)
train_df = reduce_mem_usage(train_df)

In [ ]:
# Tranform horizontal representation to vertical "view"
# Our "index" will be 'id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'
# and labels are 'd_' columns

index_columns = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

grid_df = pd.melt(train_df, 
                  id_vars = index_columns, 
                  var_name = 'd', 
                  value_name = TARGET)

# Add "test set" to our grid to make predictions
add_grid = pd.DataFrame()
for i in range(1, 29):
    temp_df = train_df[index_columns]
    temp_df = temp_df.drop_duplicates()
    temp_df['d'] = 'd_'+ str(END_TRAIN+i)
    temp_df[TARGET] = np.nan
    add_grid = pd.concat([add_grid, temp_df])

grid_df = pd.concat([grid_df, add_grid])
grid_df = grid_df.reset_index(drop=True)

# Remove some temporary dfs and original train_df
del temp_df, add_grid, train_df

# Free some memory by converting "strings" to categorical
for col in index_columns:
    grid_df[col] = grid_df[col].astype('category')

Add features: product release date

In [ ]:
# Prices are set by week so it will not have very accurate release week 
release_df = prices_df.groupby(['store_id', 'item_id'])['wm_yr_wk'].agg(['min']).reset_index()
release_df.columns = ['store_id', 'item_id', 'release']

# Merge release_df
grid_df = merge_by_concat(grid_df, release_df, ['store_id', 'item_id'])
del release_df

# Remove some "zeros" rows from grid_df 
grid_df = merge_by_concat(grid_df, calendar_df[['wm_yr_wk', 'd']], ['d'])
                      
# Cut off some rows and save memory 
grid_df = grid_df[grid_df['wm_yr_wk'] >= grid_df['release']]
grid_df = grid_df.reset_index(drop=True)

# Minify the release values.
grid_df['release'] = grid_df['release'] - grid_df['release'].min()
grid_df['release'] = grid_df['release'].astype(np.int16)

# Save BASE grid for model training
grid_df.to_pickle('grid_part_1.pkl')

Add features: price features

In [ ]:
prices_df['price_max'] = prices_df.groupby(['store_id', 'item_id'])['sell_price'].transform('max')
prices_df['price_min'] = prices_df.groupby(['store_id', 'item_id'])['sell_price'].transform('min')
prices_df['price_std'] = prices_df.groupby(['store_id', 'item_id'])['sell_price'].transform('std')
prices_df['price_mean'] = prices_df.groupby(['store_id', 'item_id'])['sell_price'].transform('mean')
prices_df['price_norm'] = prices_df['sell_price'] / prices_df['price_max']
prices_df['price_nunique'] = prices_df.groupby(['store_id', 'item_id'])['sell_price'].transform('nunique')
prices_df['item_nunique'] = prices_df.groupby(['store_id', 'sell_price'])['item_id'].transform('nunique')

calendar_prices = calendar_df[['wm_yr_wk', 'month', 'year']]
calendar_prices = calendar_prices.drop_duplicates(subset=['wm_yr_wk'])
prices_df = prices_df.merge(calendar_prices[['wm_yr_wk', 'month', 'year']], on=['wm_yr_wk'], how='left')
del calendar_prices

prices_df['price_momentum'] = prices_df['sell_price'] / prices_df.groupby(['store_id', 'item_id'])['sell_price'].transform(lambda x: x.shift(1))
prices_df['price_momentum_m'] = prices_df['sell_price'] / prices_df.groupby(['store_id', 'item_id', 'month'])['sell_price'].transform('mean')
prices_df['price_momentum_y'] = prices_df['sell_price'] / prices_df.groupby(['store_id', 'item_id', 'year'])['sell_price'].transform('mean')
del prices_df['month'], prices_df['year']

Merge prices

In [ ]:
original_columns = list(grid_df)
grid_df = grid_df.merge(prices_df, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')
keep_columns = [col for col in list(grid_df) if col not in original_columns]
grid_df = grid_df[MAIN_INDEX + keep_columns]
grid_df = reduce_mem_usage(grid_df)

# Safe part 2
grid_df.to_pickle('grid_part_2.pkl')

# We don't need prices_df anymore
del prices_df

# Load part 1
grid_df = pd.read_pickle('grid_part_1.pkl')

Merge calendar

In [ ]:
grid_df = grid_df[MAIN_INDEX]

# Merge calendar partly
icols = ['date',
         'd',
         'event_name_1',
         'event_type_1',
         'event_name_2',
         'event_type_2',
         'snap_CA',
         'snap_TX',
         'snap_WI']

grid_df = grid_df.merge(calendar_df[icols], on=['d'], how='left')

# Minify data
# 'snap_' columns we can convert to bool or int8
icols = ['event_name_1',
         'event_type_1',
         'event_name_2',
         'event_type_2',
         'snap_CA',
         'snap_TX',
         'snap_WI']
for col in icols:
    grid_df[col] = grid_df[col].astype('category')

# Convert to DateTime
grid_df['date'] = pd.to_datetime(grid_df['date'])

# Make some features from date
grid_df['tm_d'] = grid_df['date'].dt.day.astype(np.int8)
grid_df['tm_w'] = grid_df['date'].dt.week.astype(np.int8)
grid_df['tm_m'] = grid_df['date'].dt.month.astype(np.int8)
grid_df['tm_y'] = grid_df['date'].dt.year
grid_df['tm_y'] = (grid_df['tm_y'] - grid_df['tm_y'].min()).astype(np.int8)
grid_df['tm_wm'] = grid_df['tm_d'].apply(lambda x: ceil(x/7)).astype(np.int8)

grid_df['tm_dw'] = grid_df['date'].dt.dayofweek.astype(np.int8)
grid_df['tm_w_end'] = (grid_df['tm_dw']>=5).astype(np.int8)

# Remove date
del grid_df['date']

# Save part 3
grid_df.to_pickle('grid_part_3.pkl')

# We don't need calendar_df anymore
del calendar_df, grid_df

Final grid dataframe

In [ ]:
# Convert 'd' to int
grid_df = pd.read_pickle('grid_part_1.pkl')
grid_df['d'] = grid_df['d'].apply(lambda x: x[2:]).astype(np.int16)

# Remove 'wm_yr_wk' as test values are not in train set
del grid_df['wm_yr_wk']
grid_df.to_pickle('grid_part_1.pkl')

del grid_df

grid_df = pd.concat([pd.read_pickle('grid_part_1.pkl'),
                     pd.read_pickle('grid_part_2.pkl').iloc[:,2:],
                     pd.read_pickle('grid_part_3.pkl').iloc[:,2:]],
                     axis=1)

grid_df.to_pickle('grid.pkl')
for i in range(1, 4):
    os.remove('grid_part_{}.pkl'.format(str(i)))